In [546]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import datetime

import nltk
import spacy
from textblob import TextBlob

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import words
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer, MWETokenizer

from nltk.sentiment import SentimentIntensityAnalyzer, vader


from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.decomposition import LatentDirichletAllocation, NMF

from sklearn.cluster import KMeans, MiniBatchKMeans, AgglomerativeClustering, DBSCAN, MeanShift

from nltk.cluster.kmeans import KMeansClusterer
from nltk.cluster.util import cosine_distance

from sklearn import preprocessing

from sklearn import metrics

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import joblib
pd.options.display.colheader_justify = 'right'
pd.options.display.column_space = 1
pd.options.display.expand_frame_repr = True
pd.options.display.max_colwidth = 120

In [548]:
savedir = './plots'

In [38]:
df = joblib.load('data/clean/clean_df.joblib')
df = df.loc[df.body != 'full quote', :] # drop the 'full quote' only comment
df = df[~df.body.str.contains("streamable mirror")] # Drop twitter bot comments

In [3]:
# don't use stemming with Word2Vec

In [4]:
# TODO: Properly do entire data set and a train_test_split later on
# df_small = df.sample(30000, random_state=0)

# df_small_train = df_small[:int(0.7 * df_small.shape[0])]
# df_small_test = df_small[int(0.7 * df_small.shape[0]):]

# TF

### Next steps:
1. I want to find the X topics/clusters about these texts. Perhaps it's fitness, politics, training, performance, GOAT.
2. Figure out which topic each document best corresponds to, and place the document within that topic/cluster
3. Chart a stacked-bar or a multi-line chart for each year (or month, or something else) from 2011-2018, and show
   the PERCENTAGE (Or COUNT) of each topic for that period of time.
4. Look at the insights! What does that mean?
5. Perhaps do sentiment analysis. See if I can find a well-trained model. Or train my own based on upvote? score? (Probably not doing this)

**NOTE**: Simply do LDA first, and make this work. Get a chart. Get some insights.
**THEN:** Make it repeatable. Get it in a function somehow. Try other Dim Red and Unclassifying and Topic modeling techniques. See what works best.

In [39]:
def see_lda_topics(vectorizer, n_topics, fit_lda, n_examples):
    '''
    Prints top 10 names assigned to the topics, and the topics themselves
    '''
    vocab = vectorizer.get_feature_names()
    for topic in range(n_topics):
        print(f"TOPIC {topic}")
        for j in np.argsort(-fit_lda.components_,1)[topic,:n_examples]:
            print(vocab[j])
        print()

In [105]:
words_corpus = set(words.words())
analyzer = CountVectorizer().build_analyzer()
stem = SnowballStemmer('english')

stops = stopwords.words('english')
stops += ['lebron', 'james', 'game', 'us', 'need', 'let', 'well', 'year', 'dude', 'could', 'couldnt',
          'gon', 'gonna', 'na', 'ca', 'really', 'man', 'much',# 'last', 'next',
          'just', 'lol', 'like', 'im', 'he', 'hes', 'would', 'get', 'going', 'got', 'every', 'shit',
          'doesnt', 'th', 'fuck', 'think', 'even', 'dont', 'even', 'pretty', 'really', 'one',
          'didnt', 'cant', 'say', 'see', 'look', 'go', 'said', 'also', 'still', 'good'
         ]
stops = set(stops)
acceptable_words = words_corpus - stops

# create a custom vectorizer class that inherits from base class
# add a few more custom preprocessing and tokenization steps
class CustomVectorizer(CountVectorizer):  
    def build_analyzer(self):
        stop_words = stops
        
        def analyzer(doc):
            
            # preprocess: do any further cleaning here if needed
            # example: remove everything but letters
#             cleaned_doc = re.sub(r"[^A-Za-z]", " ", doc)
            cleaned_doc = doc
    
            # instantiate snowball stemmer
            stemmer = SnowballStemmer("english")
            # create tokens
            tokens = [stemmer.stem(d) for d in word_tokenize(cleaned_doc) if d in acceptable_words]
            tokens = [token for token in tokens if len(token) > 3]
            
            # multi-word expression tokenizer
            tokenizer = MWETokenizer()
            tokenizer.add_mwe(('k', 'love'))
            tokenizer.add_mwe(('kevin', 'love'))
            tokenizer.add_mwe(('dwayne', 'wade'))
            tokenizer.add_mwe(('d', 'wade'))
            tokenizer.add_mwe(('jr', 'smith'))
            tokenizer.add_mwe(('j', 'r', 'smith'))
            tokenizer.add_mwe(('j', 'r'))
            # tokenize the tokens
            tokens = tokenizer.tokenize(tokens)
            
            return(self._word_ngrams(tokens, stop_words))
        return analyzer

In [50]:
# def english_corpus(doc, stemmer=stem):
#     clean_words = [stemmer.stem(w) for w in analyzer(doc) if w in acceptable_words]   
#     return [stemmer.stem(w) for w in analyzer(doc) if w in acceptable_words]

Current isues:
* can't use stemmer and n_grams at same time.
* still too many junk words, even with growing stop_list
* **don't have a lemmatizer in there. How can i do lemmatize, n_grams, stop words, and stemmer all in analyer?**

In [51]:
# BI GRAMS

In [52]:
# cVectorizer = CustomVectorizer(# strip_accents='ascii',
#                                max_df=0.70,
#                                min_df=4,
#                                stop_words = stops,
#                                ngram_range=(2,2)
#                               )

# dtm_tf_custom = cVectorizer.fit_transform(df_small_train.body)
# #dtm_tf_custom.shape

# for n_topic in range(4,10):
#     loop_lda_tf = LatentDirichletAllocation(n_components=n_topic, random_state=0)
#     loop_lda_tf.fit(dtm_tf_custom)
#     print(':::::')
#     print(f'Number of topics = {n_topic}')
#     print(':::::')
#     see_lda_topics(cVectorizer, n_topic, loop_lda_tf, 10)

In [53]:
# MOSTLY UNI GRAMS. ALLOWS BI AND TRI GRAMS

In [67]:
cVectorizer = CustomVectorizer(# strip_accents='ascii',
                               max_df=0.7,
                               min_df=4,
                               stop_words = stops,
                               ngram_range=(1,3)
                              )

dtm_tf_custom = cVectorizer.fit_transform(df.body)
#dtm_tf_custom.shape

for n_topic in range(7,10):
    loop_lda_tf = LatentDirichletAllocation(n_components=n_topic, random_state=0, n_jobs=-1)
    loop_lda_tf.fit(dtm_tf_custom)
    print(':::::')
    print(f'Number of topics = {n_topic}')
    print(':::::')
    see_lda_topics(cVectorizer, n_topic, loop_lda_tf, 15)

:::::
Number of topics = 7
:::::
TOPIC 0
one
season
better
player
best
leagu
point
ever
shoot
year
curri
make
per
defens
time

TOPIC 1
love
time
year
play
great
coach
defens
day
one
mani
last
actual
feel
post
back

TOPIC 2
win
team
could
top
basketbal
right
time
peopl
goat
take
player
make
last
probabl
one

TOPIC 3
team
play
pick
pass
best
take
player
anyon
way
better
bad
someon
els
big
everyon

TOPIC 4
wade
peopl
guy
come
heat
want
talk
new
know
confirm
give
nice
fan
team
hate

TOPIC 5
team
year
back
first
win
season
know
one
best
come
tripl
doubl
next
trade
make

TOPIC 6
watch
damn
dude
ball
high
play
jordan
call
know
actual
time
never
k
great
foul

:::::
Number of topics = 8
:::::
TOPIC 0
one
player
best
leagu
make
ever
shoot
season
point
seen
better
defens
curri
team
time

TOPIC 1
love
time
great
year
play
coach
defens
actual
day
one
post
way
lue
last
feel

TOPIC 2
win
basketbal
top
right
could
time
team
peopl
goat
take
player
make
beat
probabl
last

TOPIC 3
best
team
play
player
p

In [55]:
# BI and TRI-GRAMS

In [68]:
cVectorizer = CustomVectorizer(# strip_accents='ascii',
                               max_df=0.9,
                               min_df=2,
                               stop_words = stops,
                               ngram_range=(2,4)
                              )

dtm_tf_custom = cVectorizer.fit_transform(df_small.body)
#dtm_tf_custom.shape

for n_topic in range(2,6):
    loop_lda_tf = LatentDirichletAllocation(n_components=n_topic, random_state=0, n_jobs=-1)
    loop_lda_tf.fit(dtm_tf_custom)
    print(':::::')
    print(f'Number of topics = {n_topic}')
    print(':::::')
    see_lda_topics(cVectorizer, n_topic, loop_lda_tf, 15)

:::::
Number of topics = 2
:::::
TOPIC 0
last year
high school
tripl doubl
ever seen
regular season
next year
golden state
eastern confer
first round
dan gilbert
free throw
never seen
best ever
win championship
last season

TOPIC 1
best player
regular season
year old
player leagu
come back
best player leagu
god damn
one best
basketbal player
player world
wade bosh
last year
locker room
first time
watch play

:::::
Number of topics = 3
:::::
TOPIC 0
high school
ever seen
last year
eastern confer
one time
anyon els
derrick rose
point guard
never seen
win championship
top player
come back
role model
best ever
space jam

TOPIC 1
best player
player leagu
come back
best player leagu
locker room
year old
god damn
wade bosh
first time
make sure
big deal
back back
player world
three point
player team

TOPIC 2
last year
tripl doubl
regular season
next year
year old
basketbal player
one best
best player
golden state
free throw
dan gilbert
last season
player ever
everyon els
first round

:::::
Num

# Run chosen LDA model, cluster, then plot

In [4]:
cVectorizer = CustomVectorizer(# strip_accents='ascii',
                               max_df=0.7,
                               min_df=4,
                               stop_words = stops,
                               ngram_range=(1,3)
                              )

dtm_tf_custom = cVectorizer.fit_transform(df.body)
#dtm_tf_custom.shape

lda = LatentDirichletAllocation(n_components=9, random_state=0, n_jobs=-1)
#loop_lda_tf.fit(dtm_tf_custom)

transformed_lda = lda.fit_transform(dtm_tf_custom)

In [71]:
pyLDAvis.sklearn.prepare(lda, dtm_tf_custom, cVectorizer)

/home/spencer/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.135299  0.041656       1        1  14.107050
2      0.027982 -0.015418       2        1  11.789902
3      0.033736  0.063955       3        1  11.603419
6     -0.073032  0.108014       4        1  11.281074
5      0.118416 -0.050655       5        1  10.775733
1     -0.080613  0.050426       6        1  10.543028
8     -0.109077  0.114184       7        1  10.400451
7      0.142663 -0.112248       8        1  10.340898
4     -0.195375 -0.199914       9        1   9.158446, topic_info=      Category         Freq       Term        Total  loglift  logprob
term                                                                 
11783  Default  2175.000000       year  2175.000000  30.0000  30.0000
1004   Default  1797.000000       best  1797.000000  29.0000  29.0000
8610   Default  1818.000000     season  1818.000000  28.0000  28.0000
1107   Default  1621.000000     better  1621.000000  27.0000  27.0000
2363   Default   835.000000       damn   835.000000  26.0000  26.0000
11246  Default  1204.000000      watch  1204.000000  25.0000  25.0000
5682   Default  1726.000000       love  1726.000000  24.0000  24.0000
6713   Default  3187.000000        one  3187.000000  23.0000  23.0000
11093  Default  1035.000000       wade  1035.000000  22.0000  22.0000
9911   Default  3042.000000       team  3042.000000  21.0000  21.0000
10580  Default   830.000000        top   830.000000  20.0000  20.0000
3574   Default  1287.000000      first  1287.000000  19.0000  19.0000
633    Default  1605.000000       back  1605.000000  18.0000  18.0000
7298   Default   535.000000       pick   535.000000  17.0000  17.0000
7451   Default  2170.000000     player  2170.000000  16.0000  16.0000
6492   Default   847.000000       next   847.000000  15.0000  15.0000
2501   Default   995.000000     defens   995.000000  14.0000  14.0000
2831   Default   681.000000       dude   681.000000  13.0000  13.0000
4003   Default   472.000000       goat   472.000000  12.0000  12.0000
11524  Default  1497.000000        win  1497.000000  11.0000  11.0000
7346   Default  1759.000000       play  1759.000000  10.0000  10.0000
848    Default  1060.000000  basketbal  1060.000000   9.0000   9.0000
8932   Default   797.000000       shot   797.000000   8.0000   8.0000
1810   Default   550.000000      coach   550.000000   7.0000   7.0000
3172   Default  1191.000000       ever  1191.000000   6.0000   6.0000
5274   Default  1266.000000       last  1266.000000   5.0000   5.0000
5053   Default   365.000000          k   365.000000   4.0000   4.0000
4968   Default  1101.000000     jordan  1101.000000   3.0000   3.0000
10358  Default  2801.000000       time  2801.000000   2.0000   2.0000
2758   Default   409.000000      doubl   409.000000   1.0000   1.0000
...        ...          ...        ...          ...      ...      ...
11093   Topic9   628.894065       wade  1035.220216   1.8921  -3.9536
4522    Topic9   131.388284        hey   176.523145   2.0952  -5.5194
4947    Topic9    98.712201     jersey   128.593658   2.1260  -5.8053
6521    Topic9   233.040145       nice   367.054591   1.9362  -4.9463
5112    Topic9   184.642264       king   285.442330   1.9549  -5.1791
10127   Topic9   240.569192       tell   419.956594   1.8334  -4.9145
1494    Topic9    84.202709          c   111.277690   2.1117  -5.9643
10564   Topic9   197.194936    tonight   338.581360   1.8499  -5.1133
6487    Topic9   118.968659       news   175.492620   2.0018  -5.6187
9858    Topic9   351.638147       talk   746.526591   1.6377  -4.5349
6270    Topic9   171.737879      money   299.682206   1.8337  -5.2515
11187   Topic9   391.039069       want   940.221166   1.5132  -4.4287
8000    Topic9   214.062773          r   429.935625   1.6931  -5.0312
5067    Topic9   204.988044       keep   446.306904   1.6124  -5.0746
1854    Topic9   381.676903       come  1210.271092   1.2365  -4.4529
422

In [202]:
# cossim = cosine_similarity(transformed_lda[:1000]) USE THIS ONE
#eucdist = euclidean_distances(transformed_lda[:20000])

# # generate cosine_similarity and euclidian distance scores. They seem quite similar. For now i'll choose cosine similarity
# print(df.body.iloc[6])
# print(df.body.iloc[7])
# print(cossim[6,7])
# print(eucdist[6,7])

# Use Clustering Algo to group documents into X num_topics by most similar topics

In [145]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=700, ngram_range=(1,3), stop_words=stops)
tfidf = tfidf_vectorizer.fit_transform(df.reset_index().body)
lda_model = LatentDirichletAllocation(n_components=16, max_iter=10, n_jobs=-1, learning_method='batch', learning_offset=50.,random_state=0).fit(tf)

# pyLDAvis.sklearn.prepare(lda_model, tf, tf_vectorizer)

/home/spencer/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
3      0.138691 -0.093932       1        1  7.819725
13    -0.095447 -0.044554       2        1  7.672248
5      0.209938  0.039630       3        1  7.667725
0      0.171528 -0.167723       4        1  7.089629
4      0.018513 -0.039297       5        1  7.051653
14    -0.106478 -0.020179       6        1  6.804064
12    -0.045867 -0.054395       7        1  6.257893
6     -0.121045 -0.102032       8        1  6.007928
9     -0.066942  0.013271       9        1  5.902155
11     0.077669  0.002877      10        1  5.797299
10    -0.069536  0.080811      11        1  5.790265
1      0.035336 -0.147373      12        1  5.715761
7     -0.252821  0.043616      13        1  5.335432
8      0.098557  0.162592      14        1  5.115834
2      0.062705  0.317898      15        1  5.111311
15    -0.054801  0.008788      16        1  4.861078, topic_info=     Category         Freq       Term        Total  loglift  logprob
term                                                                
615   Default  3094.000000       team  3094.000000  30.0000  30.0000
453   Default  2245.000000     player  2245.000000  29.0000  29.0000
371   Default  1819.000000       love  1819.000000  28.0000  28.0000
532   Default  1837.000000     season  1837.000000  27.0000  27.0000
37    Default  1624.000000       back  1624.000000  26.0000  26.0000
55    Default  1852.000000       best  1852.000000  25.0000  25.0000
669   Default  1243.000000      watch  1243.000000  24.0000  24.0000
410   Default  1321.000000      never  1321.000000  23.0000  23.0000
321   Default  1139.000000     jordan  1139.000000  22.0000  22.0000
377   Default  1658.000000       make  1658.000000  21.0000  21.0000
59    Default  1641.000000     better  1641.000000  20.0000  20.0000
514   Default  1359.000000      right  1359.000000  19.0000  19.0000
111   Default  1230.000000       come  1230.000000  18.0000  18.0000
154   Default  1030.000000     defens  1030.000000  17.0000  17.0000
259   Default  1441.000000      great  1441.000000  16.0000  16.0000
663   Default  1052.000000       wade  1052.000000  15.0000  15.0000
145   Default   858.000000       damn   858.000000  14.0000  14.0000
38    Default  1000.000000       ball  1000.000000  13.0000  13.0000
668   Default   975.000000       want   975.000000  12.0000  12.0000
471   Default   993.000000    probabl   993.000000  11.0000  11.0000
614   Default   791.000000       talk   791.000000  10.0000  10.0000
44    Default  1097.000000  basketbal  1097.000000   9.0000   9.0000
144   Default   783.000000      curri   783.000000   8.0000   8.0000
251   Default   822.000000       give   822.000000   7.0000   7.0000
213   Default   742.000000       feel   742.000000   6.0000   6.0000
183   Default   667.000000       edit   667.000000   5.0000   5.0000
611   Default  1397.000000       take  1397.000000   4.0000   4.0000
541   Default   900.000000      shoot   900.000000   3.0000   3.0000
82    Default   653.000000       call   653.000000   2.0000   2.0000
4     Default   959.000000     actual   959.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
173   Topic16   151.684270       drop   153.155104   3.0143  -4.1466
418   Topic16   101.099950     normal   102.099411   3.0141  -4.5522
233   Topic16    91.976947     forgot    92.910667   3.0138  -4.6468
99    Topic16    78.700600      chuck    79.634328   3.0121  -4.8027
644   Topic16   157.174422      trash   161.229088   2.9984  -4.1110
440   Topic16   117.226675       peak   124.437967   2.9642  -4.4042
312   Topic16    98.206655      insid   106.905008   2.9390  -4.5813
320   Topic16   154.880834       joke   172.837776   2.9142  -4.1257
253   Topic16   122.818081       glad   137.516456   2.9109  -4.3576
1     Topic16   298.855808    absolut   371.483113   2.8064  -3.4684
484   Topic16   122.187787       quo

In [ ]:
# PICK A CLUSTERING ALGO THAT USES COSINE SIMILARITY
### PERHAPS DBSCAN
##### create clusters (start with a small number, cluster_num DOES NOT need to equal the number of topics)

#----------- Chad recommended this:
# plot by argmax (basically saying which topic is most dominant in each document)
# plot by a groupby(time) and then mean the values of each topic. so then i'll have "documents for jan 2011 have [t1, t2, t3, t4, t5] and plot that.
# look into a hard clusting like Kmeans (right now, let's put this behind us)

In [248]:
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [413]:
lda_W.shape

(45657, 16)

In [409]:
lda_W[:2]

array([[0.02083333, 0.35416664, 0.02083333, 0.02083333, 0.02083333,
        0.02083333, 0.35416667, 0.02083333, 0.02083333, 0.02083333,
        0.02083333, 0.02083333, 0.02083336, 0.02083333, 0.02083333,
        0.02083333],
       [0.25881531, 0.00892858, 0.15211729, 0.00892857, 0.00892857,
        0.16842655, 0.00892859, 0.3134979 , 0.00892857, 0.00892859,
        0.00892858, 0.0089286 , 0.00892857, 0.00892857, 0.00892858,
        0.00892857]])

In [412]:
lda_topic_index = []
for i in lda_W:
    lda_topic_index.append(np.argmax(i))
len(lda_topic_index)

45657

In [415]:
lda_topic_index[0:13]

[6, 7, 13, 7, 0, 5, 2, 2, 7, 5, 4, 1, 5]

In [422]:
lda_topics_df = pd.DataFrame(lda_topic_index, copy=True, columns=['topics'])
lda_topics_df['time'] = df.reset_index()['created_utc']

# Pickle Models for plots in other notebook

In [795]:
model_dir = './pickled_models'
joblib.dump(lda_W, f'{model_dir}/lda_W.joblib')
joblib.dump(lda_topics_df, f'{model_dir}/lda_topics_df.joblib')